In [13]:
import os
import time
import requests
from pathlib import Path
import pandas as pd
from datetime import datetime
from google.transit import gtfs_realtime_pb2
from dotenv import load_dotenv
print("Import success!")

Import success!


In [7]:
print(Path("/workspace/.env").exists())
# --- cél elérési út a /workspace/data alá ---
data_dir = Path("/workspace/data")
data_dir.mkdir(parents=True, exist_ok=True)

csv_path = data_dir / "test_file.csv"

# --- példaadatok ---
df = pd.DataFrame({
    "id": [1, 2, 3],
    "name": ["bus", "tram", "metro"],
    "speed_kmh": [45, 60, 80]
})

# --- mentés ---
df.to_csv(csv_path, index=False)
print(f"✅ Mentve: {csv_path}")

# --- visszaellenőrzés ---
print("Létezik?", csv_path.exists())
print(df)

True
✅ Mentve: /workspace/data/test_file.csv
Létezik? True
   id   name  speed_kmh
0   1    bus         45
1   2   tram         60
2   3  metro         80


In [ ]:
BASE_DIR = Path(__file__).resolve().parent
load_dotenv(dotenv_path=BASE_DIR / ".env")

API_KEY = os.getenv("BKK_API_KEY")
if not API_KEY:
    raise RuntimeError("API key not found in .env file (BKK_API_KEY=...)")

# --- Adatmappa ---
DATA_DIR = BASE_DIR / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)
DATA_PATH = DATA_DIR / "vehicle_updates.csv"

# --- API URL-ek ---
URL_POS = f"https://go.bkk.hu/api/query/v1/ws/gtfs-rt/full/VehiclePositions.pb?key={API_KEY}"
URL_TRIP = f"https://go.bkk.hu/api/query/v1/ws/gtfs-rt/full/TripUpdates.pb?key={API_KEY}"

print("=== BKK FUTÁR tesztadatgyűjtés (lokális mód) ===")
print(f"VehiclePositions: {URL_POS}")
print(f"TripUpdates: {URL_TRIP}")
print(f"Mentés: {DATA_PATH}")



for _ in range(2):
    try:
        #  Pozíciók lekérése
        res_pos = requests.get(URL_POS, headers={"Accept": "application/x-protobuf"}, timeout=10)
        res_pos.raise_for_status()

        feed_pos = gtfs_realtime_pb2.FeedMessage()
        feed_pos.ParseFromString(res_pos.content)

        #  Késések lekérése
        res_trip = requests.get(URL_TRIP, headers={"Accept": "application/x-protobuf"}, timeout=10)
        res_trip.raise_for_status()

        feed_trip = gtfs_realtime_pb2.FeedMessage()
        feed_trip.ParseFromString(res_trip.content)

        # késés dictionary (trip_id -> delay)
        delay_map = {}
        for ent in feed_trip.entity:
            if hasattr(ent, "trip_update") and hasattr(ent.trip_update.trip, "trip_id"):
                trip_id = ent.trip_update.trip.trip_id
                if len(ent.trip_update.stop_time_update) > 0:
                    delay = getattr(ent.trip_update.stop_time_update[0].arrival, "delay", None)
                    if delay is not None:
                        delay_map[trip_id] = delay

        now = datetime.utcnow()
        records = []

        for entity in feed_pos.entity:
            v = entity.vehicle
            trip_id = getattr(v.trip, "trip_id", None)
            delay_val = delay_map.get(trip_id, None)

            records.append({
                "timestamp_utc": now.strftime("%Y-%m-%d %H:%M:%S"),
                "vehicle_id": entity.id,
                "trip_id": trip_id,
                "route_id": getattr(v.trip, "route_id", None),
                "lat": getattr(v.position, "latitude", None),
                "lon": getattr(v.position, "longitude", None),
                "speed": getattr(v.position, "speed", None),
                "heading": getattr(v.position, "bearing", None),
                "vehicle_timestamp": datetime.fromtimestamp(v.timestamp),
                "last_stop_id": getattr(v, "stop_id", None),
                "delay_seconds": delay_val,
            })

        df = pd.DataFrame(records)
        print(df.head(10)[["trip_id", "last_stop_id", "delay_seconds"]])  # csak a lényeg
        df.to_csv(DATA_PATH, mode="a", header=not DATA_PATH.exists(), index=False)
        print(f"{len(records)} rekord mentve {now.strftime('%H:%M:%S')}\n")

    except Exception as e:
        print("Hiba:", e)

    time.sleep(60)  # TEST

NameError: name '__file__' is not defined

In [ ]:
DATA_PATH
